# Decision Analysis Project 1

## Installs and Imports

In [92]:
#Installs go here

In [93]:
#Imports go here
import warnings
import numpy as np
import pandas as pd

## Utilities

In [94]:
def calculate_value(value, q, p):
    if value > p:
        return 1.0
    elif value<=q:
        return 0
    return (value-q)/(p-q)

class Criteria:
    '''
    Criteria class for PROMETHEE and ELECTRE implementations.
    '''
    def __init__(self, name: str, criteria_type: str, w: float, q: float, p:float, v:float = float("inf")):
        '''
        Initialize instance of Criteria.
        Parameters:
            - name (str): Criteria name, e.g. (cost, completion time, rating, etc.)
            - criteria_type (str): Criteria type - 'gain' (higher values preffered) or 'cost' (lower values preffered)
            - w (float): Criteria weight
            - q (float): Indifference threshold
            - p (float): Perference threshold
            - v (float, optional): Veto threshold, default infinite -> used only for ELECTRE
        '''
        self.name = name
        assert criteria_type.lower() == "gain" or criteria_type.lower() == "cost", "Invalid criteria type"
        assert q<=p and p<=v and p>=0 and q>=0 and w>=0, "Invalid values, expected: q<p<v and q,p,w >=0"
        assert not np.isposinf(w) and not np.isneginf(w), "Invalid value, weight cannot be infinite"
        self.criteria_type = criteria_type.lower()
        self.w = w
        self.q = q
        self.p = p
        self.v = v

    def __str__(self):
        '''
        Print information about Criteria.
        '''
        return f"Criteria: {self.name} \n type: {self.criteria_type} \n w: {self.w} \n q: {self.q} \n p: {self.p} \n v: {self.v} \n"

## PROMETHEE

In [95]:
class PROMETHEE:
    def __init__(self, name: str = "MCDA Problem", criteria: list = [], alternatives = None, initialize_from_input: bool = False, initialize_from_file: bool = False, criteria_path = "", alternatives_path = "", version: int = 1):
        if alternatives is not None and not isinstance(alternatives, pd.DataFrame):
            raise TypeError("Alternatives must be a pandas DataFrame.")

        if not initialize_from_input and not initialize_from_file and len(criteria) == 0:
            warnings.warn("Criteria empty but idata from file or input is not read >:(", UserWarning)

        self.name = name
        if version != 1 and version != 2:
            raise ValueError("Only supported versions are 1 and 2")
        self.version = version

        if initialize_from_file:
            self.initalize_data_from_file(criteria_path, alternatives_path, criteria)

        elif initialize_from_input:
            self.initalize_data_from_input()

        else:
            self.criteria = criteria
            self.alternatives = alternatives

    def initalize_data_from_input(self):
        print("\n-----------------Initalize Criteria----------------\n")
        num_criteria = int(input("Num criteria = "))
        criterias = []
        for i in range(num_criteria):
            name = str(input("Name: "))
            cr_type = str(input("Type: "))
            w = int(input("Weight: "))
            q = int(input("Indifference Threshold: "))
            p = int(input("Preference Threshold: "))
            criterias.append(Criteria(name, cr_type, w, q, p))
        self.criteria = criterias

        print("\n------------------Initalize Alternatives--------------\n")
        num_alternatives = int(input("Num alternatives = "))
        alternatives = {criterias[i].name: [] for i in range(len(criterias))}
        alternatives["name"] = []
        for i in range(num_alternatives):
            name = str(input("ALternative Name: "))
            alternatives["name"].append(name)
            for ii in range(num_criteria):
                value = float(input(f"{criterias[ii].name} value: "))
                alternatives[criterias[ii].name].append(value)
        alternatives = pd.DataFrame.from_dict(alternatives)
        self.alternatives = alternatives

    def initalize_data_from_file(self, criteria_path, alternatives_path, criteria):
        if criteria_path == "":
            self.criteria = criteria
        else:
            self.criteria = pd.read_csv(criteria_path)
        self.alternatives = pd.read_csv(alternatives_path)

    def __str__(self):
        promethee_text = f"PROMETHEE {self.name}, version: {self.version}"
        for criterion in self.criteria:
            cr_printed = criterion.__str__()
            promethee_text += cr_printed
        promethee_text += "\n-----------------------Alternatives-----------------------\n"
        promethee_text += self.alternatives.__str__()
        return promethee_text

    def display(self):
        promethee_text = f"PROMETHEE {self.name}, version: {self.version}"
        for criterion in self.criteria:
            cr_printed = criterion.__str__()
            promethee_text += cr_printed
        promethee_text += "\n-----------------------Alternatives-----------------------\n"
        print(promethee_text)
        return self.alternatives.head(len(self.alternatives))

    def get_marginal_preference_index(self):
        marginal_index = {name: None for name in self.alternatives["name"]}
        print(marginal_index)
        copied_alts = self.alternatives.drop(columns=["name"])
        for i, row in copied_alts.iterrows():
            diff = copied_alts.sub(row)
            for crt in self.criteria:
                if crt.criteria_type == "cost":
                    diff[crt.name] *= -1
                diff[crt.name] = diff[crt.name].apply(calculate_value, args=(crt.q, crt.p))
            diff["name"] = self.alternatives["name"]
            marginal_index[self.alternatives["name"][i]]=diff
        return marginal_index

    def solve(self):
        marginal_preference_index = self.get_marginal_preference_index()
        return marginal_preference_index

In [96]:
criteria = [
    Criteria(name = "price", criteria_type = "cost", w = 2.0, q = 3.0, p=9.0),
    Criteria(name = "critic_score", criteria_type = "gain", w = 1.0, q = 0.0, p=1.0),
    Criteria(name = "user_score", criteria_type = "gain", w = 5.0, q = 1.0, p=2.0),
    Criteria(name = "length", criteria_type = "gain", w = 6.0, q = 5.0, p=20.0),
    Criteria(name = "genres", criteria_type = "gain", w = 2.5, q = 9.0, p=15.0),
    Criteria(name = "num_of_achievements", criteria_type = "gain", w = 8.0, q = 3.3, p=92.0),
]

In [97]:
promete = PROMETHEE(initialize_from_file=True, criteria = criteria, alternatives_path = "/content/data.csv")

In [98]:
print(promete)

PROMETHEE MCDA Problem, version: 1Criteria: price 
 type: cost 
 w: 2.0 
 q: 3.0 
 p: 9.0 
 v: inf 
Criteria: critic_score 
 type: gain 
 w: 1.0 
 q: 0.0 
 p: 1.0 
 v: inf 
Criteria: user_score 
 type: gain 
 w: 5.0 
 q: 1.0 
 p: 2.0 
 v: inf 
Criteria: length 
 type: gain 
 w: 6.0 
 q: 5.0 
 p: 20.0 
 v: inf 
Criteria: genres 
 type: gain 
 w: 2.5 
 q: 9.0 
 p: 15.0 
 v: inf 
Criteria: num_of_achievements 
 type: gain 
 w: 8.0 
 q: 3.3 
 p: 92.0 
 v: inf 

-----------------------Alternatives-----------------------
                             name  price  critic_score  user_score  length  \
0          Dark Souls: Remastered    150            84          83      44   
1                  Dark Souls III    200            89          90      49   
2                        Terraria     46            81          81     102   
3                 Baldur's Gate 3    250            96          89     107   
4                  Dave the Diver     92            90          83      32   
5          

In [99]:
promete.display()

PROMETHEE MCDA Problem, version: 1Criteria: price 
 type: cost 
 w: 2.0 
 q: 3.0 
 p: 9.0 
 v: inf 
Criteria: critic_score 
 type: gain 
 w: 1.0 
 q: 0.0 
 p: 1.0 
 v: inf 
Criteria: user_score 
 type: gain 
 w: 5.0 
 q: 1.0 
 p: 2.0 
 v: inf 
Criteria: length 
 type: gain 
 w: 6.0 
 q: 5.0 
 p: 20.0 
 v: inf 
Criteria: genres 
 type: gain 
 w: 2.5 
 q: 9.0 
 p: 15.0 
 v: inf 
Criteria: num_of_achievements 
 type: gain 
 w: 8.0 
 q: 3.3 
 p: 92.0 
 v: inf 

-----------------------Alternatives-----------------------



,name,price,critic_score,user_score,length,genres,num_of_achievements
0,Dark Souls: Remastered,150,84,83,44,11,41
1,Dark Souls III,200,89,90,49,11,43
2,Terraria,46,81,81,102,9,115
3,Baldur's Gate 3,250,96,89,107,8,54
4,Dave the Diver,92,90,83,32,7,43
5,Rust,153,69,65,37,10,92
6,Hollow Knight,68,90,91,42,10,63
7,Portal 2,46,95,89,14,7,51
8,Vampire Survivors,20,86,83,25,9,204
9,Hades,115,93,88,49,9,49


In [100]:
result = promete.solve()

{'Dark Souls: Remastered': None, 'Dark Souls III': None, 'Terraria': None, "Baldur's Gate 3": None, 'Dave the Diver': None, 'Rust': None, 'Hollow Knight': None, 'Portal 2': None, 'Vampire Survivors': None, 'Hades': None, 'Subnautica': None, 'Dishonored': None, 'Ori and the Will of the Wisps': None, 'Inside': None, 'The Forest': None, 'Skyrim': None, 'Teardown': None, 'Dying Light': None, 'Enter the Gungeon': None, 'Payday 3': None, 'Kao the Kangaroo': None, "Assassin's Creed Unity": None, 'Trials Fusion': None, 'The Sims 3': None, 'Titan Souls': None}


In [105]:
result["Rust"]

,price,critic_score,user_score,length,genres,num_of_achievements,name
0,0.0,1.0,1.0,0.133333,0,0.000000,Dark Souls: Remastered
1,0.0,1.0,1.0,0.466667,0,0.000000,Dark Souls III
2,1.0,1.0,1.0,1.000000,0,0.222097,Terraria
3,0.0,1.0,1.0,1.000000,0,0.000000,Baldur's Gate 3
4,1.0,1.0,1.0,0.000000,0,0.000000,Dave the Diver
5,0.0,0.0,0.0,0.000000,0,0.000000,Rust
6,1.0,1.0,1.0,0.000000,0,0.000000,Hollow Knight
7,1.0,1.0,1.0,0.000000,0,0.000000,Portal 2
8,1.0,1.0,1.0,0.000000,0,1.000000,Vampire Survivors
9,1.0,1.0,1.0,0.466667,0,0.000000,Hades


In [103]:
result["Dark Souls III"]

,price,critic_score,user_score,length,genres,num_of_achievements,name
0,1.0,0.0,0,0.000000,0,0.000000,Dark Souls: Remastered
1,0.0,0.0,0,0.000000,0,0.000000,Dark Souls III
2,1.0,0.0,0,1.000000,0,0.774521,Terraria
3,0.0,1.0,0,1.000000,0,0.086809,Baldur's Gate 3
4,1.0,1.0,0,0.000000,0,0.000000,Dave the Diver
5,1.0,0.0,0,0.000000,0,0.515220,Rust
6,1.0,1.0,0,0.000000,0,0.188275,Hollow Knight
7,1.0,1.0,0,0.000000,0,0.052988,Portal 2
8,1.0,0.0,0,0.000000,0,1.000000,Vampire Survivors
9,1.0,1.0,0,0.000000,0,0.030440,Hades


In [104]:
result["Terraria"]

,price,critic_score,user_score,length,genres,num_of_achievements,name
0,0.0,1.0,1.0,0.000000,0,0.000000,Dark Souls: Remastered
1,0.0,1.0,1.0,0.000000,0,0.000000,Dark Souls III
2,0.0,0.0,0.0,0.000000,0,0.000000,Terraria
3,0.0,1.0,1.0,0.000000,0,0.000000,Baldur's Gate 3
4,0.0,1.0,1.0,0.000000,0,0.000000,Dave the Diver
5,0.0,0.0,0.0,0.000000,0,0.000000,Rust
6,0.0,1.0,1.0,0.000000,0,0.000000,Hollow Knight
7,0.0,1.0,1.0,0.000000,0,0.000000,Portal 2
8,1.0,1.0,1.0,0.000000,0,0.966178,Vampire Survivors
9,0.0,1.0,1.0,0.000000,0,0.000000,Hades


## SRF

## ELECTRE

## Use